## Objective:
- The objective from this project is to prepare text data with **`NLP techniques`** and  create a <b>Spam filter using NaiveBayes classifier</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

### Create a spark session and import the required libraries

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

In [26]:
from pyspark.sql.functions import *
import pyspark.sql.functions as fn
from pyspark.ml.feature import CountVectorizer,StringIndexer,RegexTokenizer,StopWordsRemover,IDF ,HashingTF ,VectorAssembler , Word2Vec ,MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes , MultilayerPerceptronClassifier , RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

### Read the data into a DataFrame

In [3]:
df = spark.read.option("delimiter", "	").csv('SMSSpamCollection')
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [4]:
df_rename = df.withColumnRenamed('_c0' , 'class').withColumnRenamed('_c1' , 'text')
df_rename.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



In [5]:
df_rename.show(10 ,False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

In [6]:
# Create a new feature column contains the length of the text column
df_length = df_rename.withColumn('length' , fn.length(col('text')))
df_length.show(10)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
+-----+--------------------+------+
only showing top 10 rows



In [7]:
# Get the average text length for each class
df_length.groupby('class').avg('length').show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [8]:
# Check Null values
df_length.select([fn.count(fn.when(fn.isnull(c), c)).alias(c) for c in df_length.columns]).show()

+-----+----+------+
|class|text|length|
+-----+----+------+
|    0|   0|     0|
+-----+----+------+



In [9]:
# Check Duplicates
grouped_df = df_length.groupBy(df_length.columns).count()
duplicate_records = grouped_df.filter(col("count") > 1)
duplicate_records.show()

+-----+--------------------+------+-----+
|class|                text|length|count|
+-----+--------------------+------+-----+
| spam|Congrats! 1 year ...|   160|    2|
|  ham|God picked up a f...|   128|    2|
|  ham|Did u got that pe...|    28|    3|
| spam|18 days to Euro20...|   135|    2|
|  ham|Whatsup there. Do...|    35|    2|
|  ham|Reverse is cheati...|    45|    2|
| spam|Want to funk up u...|   155|    2|
|  ham|When people see m...|   148|    2|
| spam|FREE for 1st week...|   158|    3|
|  ham|No no. I will che...|    46|    3|
|  ham|                 Ok.|     3|    4|
|  ham|Gud ni8 dear..slp...|    53|    2|
|  ham|Night has ended f...|   156|    3|
| spam|WIN: We have a wi...|   132|    2|
|  ham|No calls..message...|    32|    3|
|  ham|Good morning prin...|    35|    2|
|  ham|If you r @ home t...|    43|    2|
|  ham|Today is ACCEPT D...|   156|    3|
| spam|Natalja (25/F) is...|   136|    2|
|  ham|Love isn't a deci...|   126|    2|
+-----+--------------------+------

In [10]:
df_length.count()

5574

In [11]:
#Remove Duplicates
df_duplicates = df_length.dropDuplicates()
df_duplicates.count()

5171

## Feature Transformations

##### Filtering data

In [12]:
#Starting with remove anything not a letter
#df_filtered = df_duplicates.withColumn('text_filtered' , fn.regexp_replace(col('text') , '[^A-Za-z ]+', ''))

#Remove any digits from text 
#df_filtered = df_filtered.withColumn('text_filtered' , fn.regexp_replace(col('text_filtered') , '\d+', ''))

#Remove any multiple spaces
#df_filtered = df_filtered.withColumn("text_filtered",regexp_replace(col('text_filtered'), ' +', ' '))
df_filtered = df_duplicates
df_filtered.show(3,False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------+------+
|class|text                                                                                                                                  |length|
+-----+--------------------------------------------------------------------------------------------------------------------------------------+------+
|ham  |Did you catch the bus ? Are you frying an egg ? Did you make a tea? Are you eating your mom's left over dinner ? Do you feel my Love ?|134   |
|ham  |I see a cup of coffee animation                                                                                                       |31    |
|ham  |hanks lotsly!                                                                                                                         |13    |
+-----+---------------------------------------------------------------------------------------------

#### Most of spam sms have digits and punctuation so decide to keep it

#### Next steps
##### 1)transform text into tokens and remove stop words
##### 2) Apply count Vectorizer then idf to transform each word to vectors 

In [13]:
#Tokenization
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

#Remove Stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

#Apply CountVectorizer to get term frequencies
cv = CountVectorizer(inputCol="filtered", outputCol="vectorized")

# Apply IDF to get TF-IDF features
idf = IDF(inputCol="vectorized", outputCol="feature")

Convert the <b>class column</b> to index using <b>StringIndexer</b> then create <b>features Column</b>

In [14]:
#Convert column class to 0,1
indexer = StringIndexer(inputCol="class", outputCol="label")

#Using Vector Assmbeler to creater features column
vectorAssm = VectorAssembler(inputCols=['feature','length'] , outputCol='features')

#### Create Navie Bayes Model

In [15]:
nv = NaiveBayes()

#### Pipeline
##### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [16]:
stages = [regexTokenizer , remover , cv , idf , indexer ,vectorAssm ,nv]
pl = Pipeline(stages=stages)

#### Splitting data into train and test

In [19]:
train_data, test_data = df_filtered.randomSplit([0.7, 0.3], seed=42)
print(f"There are {train_data.count()} rows in the training set, and {test_data.count()} in the test set")

There are 3680 rows in the training set, and 1491 in the test set


#### Fitting Pipeline model to train data

In [20]:
model = pl.fit(train_data)
train_pred = model.transform(train_data)
train_pred.select('features' , 'label','prediction','probability').show(5)

+--------------------+-----+----------+--------------------+
|            features|label|prediction|         probability|
+--------------------+-----+----------+--------------------+
|(7056,[7,9,3633,4...|  0.0|       0.0|[1.0,1.0460178600...|
|(7056,[7,9,31,105...|  0.0|       0.0|[1.0,7.8828700561...|
|(7056,[502,783,48...|  0.0|       0.0|[0.99999990788599...|
|(7056,[10,12,68,1...|  0.0|       0.0|[1.0,3.0612301226...|
|(7056,[16,19,89,9...|  0.0|       0.0|[1.0,1.1236961980...|
+--------------------+-----+----------+--------------------+
only showing top 5 rows



24/05/11 21:28:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


#### Perform predictions on tests dataframe and evaluate using F1_score

In [21]:
test_pred = model.transform(test_data)
test_pred.select('features' , 'label','prediction','probability').show(5)

+--------------------+-----+----------+--------------------+
|            features|label|prediction|         probability|
+--------------------+-----+----------+--------------------+
|(7056,[3,7,9,18,1...|  0.0|       0.0|[1.0,6.4243448789...|
|(7056,[19,96,219,...|  0.0|       0.0|[1.0,3.3821504127...|
|(7056,[0,91,244,5...|  0.0|       1.0|[1.53625302005278...|
|(7056,[7,9,370,89...|  0.0|       0.0|[1.0,1.5204975380...|
|(7056,[0,1,6,26,3...|  0.0|       0.0|[1.0,2.9858067624...|
+--------------------+-----+----------+--------------------+
only showing top 5 rows



In [22]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(test_pred)
print("F1 Score:", f1_score)

F1 Score: 0.9765973726873007


#### Trying Randomforest model

In [23]:
rf = RandomForestClassifier(maxDepth=20 , numTrees=50)

In [24]:
stages = [regexTokenizer , remover , cv , idf , indexer,vectorAssm ,rf]
pl = Pipeline(stages=stages)


model = pl.fit(train_data)

24/05/11 21:29:05 WARN DAGScheduler: Broadcasting large task binary with size 1021.4 KiB
24/05/11 21:29:05 WARN DAGScheduler: Broadcasting large task binary with size 1052.4 KiB
24/05/11 21:29:05 WARN DAGScheduler: Broadcasting large task binary with size 1088.2 KiB
24/05/11 21:29:06 WARN DAGScheduler: Broadcasting large task binary with size 1118.4 KiB
24/05/11 21:29:06 WARN DAGScheduler: Broadcasting large task binary with size 1151.5 KiB
24/05/11 21:29:06 WARN DAGScheduler: Broadcasting large task binary with size 1183.3 KiB
24/05/11 21:29:06 WARN DAGScheduler: Broadcasting large task binary with size 1217.5 KiB
24/05/11 21:29:07 WARN DAGScheduler: Broadcasting large task binary with size 1256.2 KiB
24/05/11 21:29:07 WARN DAGScheduler: Broadcasting large task binary with size 1293.3 KiB
24/05/11 21:29:07 WARN DAGScheduler: Broadcasting large task binary with size 1333.5 KiB
24/05/11 21:29:08 WARN DAGScheduler: Broadcasting large task binary with size 1369.9 KiB
24/05/11 21:29:08 WAR

In [25]:
test_pred = model.transform(test_data)
f1_score = evaluator.evaluate(test_pred)
print("F1 Score:", f1_score)

24/05/11 21:29:48 WARN DAGScheduler: Broadcasting large task binary with size 1120.7 KiB


F1 Score: 0.9455975586448901


#### let's try with kfold cross validation 

In [27]:
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [50]) \
    .addGrid(rf.maxDepth, [30]) \
    .build()


In [28]:
cv = CrossValidator(estimator=pl,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=5)


In [29]:
cvModel1 = cv.fit(train_data)

24/05/11 21:31:22 WARN DAGScheduler: Broadcasting large task binary with size 1005.6 KiB
24/05/11 21:31:27 WARN DAGScheduler: Broadcasting large task binary with size 1037.1 KiB
24/05/11 21:31:31 WARN DAGScheduler: Broadcasting large task binary with size 1073.3 KiB
24/05/11 21:31:35 WARN DAGScheduler: Broadcasting large task binary with size 1109.8 KiB
24/05/11 21:31:40 WARN DAGScheduler: Broadcasting large task binary with size 1143.0 KiB
24/05/11 21:31:44 WARN DAGScheduler: Broadcasting large task binary with size 1180.7 KiB
24/05/11 21:31:48 WARN DAGScheduler: Broadcasting large task binary with size 1217.1 KiB
24/05/11 21:31:52 WARN DAGScheduler: Broadcasting large task binary with size 1255.9 KiB
24/05/11 21:31:56 WARN DAGScheduler: Broadcasting large task binary with size 1294.7 KiB
24/05/11 21:32:00 WARN DAGScheduler: Broadcasting large task binary with size 1330.1 KiB
24/05/11 21:32:04 WARN DAGScheduler: Broadcasting large task binary with size 1365.8 KiB
24/05/11 21:32:08 WAR

In [30]:
test_pred = cvModel1.transform(test_data)
f1_score = evaluator.evaluate(test_pred)
print("F1 Score:", f1_score)

F1 Score: 0.962274061541674


24/05/11 21:42:00 WARN DAGScheduler: Broadcasting large task binary with size 1315.2 KiB


### Lets try Word2vec with naive bayes

In [31]:
#Tokenization
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

#Remove Stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

# Word2Vec
word2Vec = Word2Vec(inputCol="filtered", outputCol="vectorized", vectorSize=10, minCount=0)

# W2Vec Dataframes typically has negative values so we will correct for that here so that we can use the Naive Bayes classifier
scaler = MinMaxScaler(inputCol="vectorized", outputCol="feature")


#Convert column class to 0,1
indexer = StringIndexer(inputCol="class", outputCol="label")

#Using Vector Assmbeler to creater features column
vectorAssm = VectorAssembler(inputCols=['feature','length'] , outputCol='features')

#Creating model
nv = NaiveBayes()

# Creating pipeline with word2vec instead of tf-idf
stages = [regexTokenizer , remover  , word2Vec ,scaler , indexer , vectorAssm , nv ]
pl = Pipeline(stages=stages)

In [32]:
model = pl.fit(train_data)
train_pred = model.transform(train_data)

In [33]:
test_pred = model.transform(test_data)
test_pred.cache()
evaluator.evaluate(test_pred)

0.8740710998140007

### as we see still tf-idf better than word2vec